In [54]:
import pymongo
from pprint import pprint
import pandas as pd
import numpy as np
from IPython.display import Image

connection =pymongo.MongoClient("localhost",27017)
db = connection.python

for i in db.list_collection_names():
    print(i)

posts
sudo
fs.files
chat
sales
login
funds
users
limited
fs.chunks
nei
airbnb
res


In [56]:
#reference 
for i in db.airbnb.find().limit(1):
    pprint(i)

{'_id': '10021707',
 'access': '',
 'accommodates': 1,
 'address': {'country': 'United States',
             'country_code': 'US',
             'government_area': 'Bushwick',
             'location': {'coordinates': [-73.93615, 40.69791],
                          'is_location_exact': True,
                          'type': 'Point'},
             'market': 'New York',
             'street': 'Brooklyn, NY, United States',
             'suburb': 'Brooklyn'},
 'amenities': ['Internet',
               'Wifi',
               'Air conditioning',
               'Kitchen',
               'Buzzer/wireless intercom',
               'Heating',
               'Smoke detector',
               'Carbon monoxide detector',
               'Essentials',
               'Lock on bedroom door'],
 'availability': {'availability_30': 0,
                  'availability_365': 0,
                  'availability_60': 0,
                  'availability_90': 0},
 'bathrooms': Decimal128('1.5'),
 'bed_type': 'Real 

#### \$group

In [57]:
#country코드가 얼마나 유니크한가 알아보자? -> group

for i in db.airbnb.aggregate([{"$match" :{"address.country_code":{"$ne":"US"}}},
                             {"$group":{"_id":{"CountryName":"$address.country"}}}]):
    pprint(i)

{'_id': {'CountryName': 'Hong Kong'}}
{'_id': {'CountryName': 'Turkey'}}
{'_id': {'CountryName': 'Spain'}}
{'_id': {'CountryName': 'Portugal'}}
{'_id': {'CountryName': 'Brazil'}}
{'_id': {'CountryName': 'Canada'}}
{'_id': {'CountryName': 'China'}}
{'_id': {'CountryName': 'Australia'}}


#### \$filter

In [58]:
#리뷰숫자가 5개이상 되는걸 뽑아보자. 
#method1 
for i in db.airbnb.aggregate([{"$match":{"reviews":{"$exists":True}}},
                              {"$match":{"$expr":{"$gt": [{"$size":"$reviews"},5]}}},
                              {"$limit":1}]):
    pprint(i)

{'_id': '10038496',
 'access': 'todo o espaço.',
 'accommodates': 4,
 'address': {'country': 'Brazil',
             'country_code': 'BR',
             'government_area': 'Copacabana',
             'location': {'coordinates': [-43.190849194463404,
                                          -22.984339360067814],
                          'is_location_exact': False,
                          'type': 'Point'},
             'market': 'Rio De Janeiro',
             'street': 'Rio de Janeiro, Rio de Janeiro, Brazil',
             'suburb': 'Copacabana'},
 'amenities': ['TV',
               'Cable TV',
               'Internet',
               'Wifi',
               'Air conditioning',
               'Kitchen',
               'Paid parking off premises',
               'Smoking allowed',
               'Doorman',
               'Elevator',
               'Buzzer/wireless intercom',
               'Family/kid friendly',
               'Washer',
               'Fire extinguisher',
               

In [59]:
#리뷰숫자가 5개이상 되는걸 뽑아보자. ($match $expr 사용 )

for i in db.airbnb.aggregate([{"$match":{"reviews":{"$exists":True}}},
                              {"$match":{"$expr":{"$gt": [{"$size":"$reviews"},5]}}},
                              {"$limit":1}]):
    pprint(i)

{'_id': '10038496',
 'access': 'todo o espaço.',
 'accommodates': 4,
 'address': {'country': 'Brazil',
             'country_code': 'BR',
             'government_area': 'Copacabana',
             'location': {'coordinates': [-43.190849194463404,
                                          -22.984339360067814],
                          'is_location_exact': False,
                          'type': 'Point'},
             'market': 'Rio De Janeiro',
             'street': 'Rio de Janeiro, Rio de Janeiro, Brazil',
             'suburb': 'Copacabana'},
 'amenities': ['TV',
               'Cable TV',
               'Internet',
               'Wifi',
               'Air conditioning',
               'Kitchen',
               'Paid parking off premises',
               'Smoking allowed',
               'Doorman',
               'Elevator',
               'Buzzer/wireless intercom',
               'Family/kid friendly',
               'Washer',
               'Fire extinguisher',
               

In [60]:
#method2 - size project하기. 

for i in db.airbnb.aggregate([{"$match":{"reviews":{"$exists":True}}},
                             {"$project":{
                                 "_id":1,
                                 "hostname":"$host.host_name",
                                 "number_of_reviews":{"$size":"$reviews"}}},
                             {"$limit":10}
                             ]):
    pprint(i)

{'_id': '10021707', 'hostname': 'Josh', 'number_of_reviews': 1}
{'_id': '10009999', 'hostname': 'Ynaie', 'number_of_reviews': 0}
{'_id': '10030955', 'hostname': 'Livia', 'number_of_reviews': 0}
{'_id': '10047964', 'hostname': 'Zeynep', 'number_of_reviews': 1}
{'_id': '10057447', 'hostname': 'Konstantin', 'number_of_reviews': 0}
{'_id': '10051164', 'hostname': 'Beatriz', 'number_of_reviews': 1}
{'_id': '10057826', 'hostname': 'Mae', 'number_of_reviews': 5}
{'_id': '10059244', 'hostname': 'Caro', 'number_of_reviews': 0}
{'_id': '10059872', 'hostname': 'Giovanni', 'number_of_reviews': 3}
{'_id': '10066928', 'hostname': 'Margaux', 'number_of_reviews': 0}


In [70]:
for i in db.airbnb.aggregate([{"$project":{"num_review":{"$filter":{
                            "input":"$reviews",
                            "as":"review",
                            "cond":{"$lte":[{"$size":"$reviews"},3]}}}}},
                             {"$match":{"num_review":{"$ne":[]}}},
                             {"$limit":1}
                             ]):
    pprint(i)

{'_id': '10021707',
 'num_review': [{'_id': '61050713',
                 'comments': 'Josh was out of town during my 1 month stay. His '
                             'roommates greeted and helped get me settled. '
                             'They were great hosts and all around cool '
                             "people. I'm a Brooklynite, but have never lived "
                             'in Bushwick.\r\n'
                             "If you're looking for an hip, authentic, and "
                             'convenient Brooklyn experience, this spot is for '
                             'you.  You can literally see the Subway platform '
                             "from Josh's window. Also a couple steps away "
                             'from anything you could possibly need... '
                             'restaurants, juice bar, organic grocery, etc. ',
                 'date': datetime.datetime(2016, 1, 31, 5, 0),
                 'listing_id': '10021707',
           

In [74]:

for i in db.funds.aggregate([{"$project":{"rounds_over_8000000":{"$filter":
                                                                 {"input":"$funding_rounds",
                                                                 "as":"round",
                                                                 "cond":{"$gte":["$$round.raised_amount",8000000]}}}}},
                            {"$match":{"rounds_over_8000000":{"$ne":[]}}}]):

    pprint(i)

    

{'_id': ObjectId('61167af198608fe75d51477e'),
 'rounds_over_8000000': [{'raised_amount': 9297130, 'round_code': 'd'}]}
{'_id': ObjectId('61167af198608fe75d51477f'),
 'rounds_over_8000000': [{'raised_amount': 9737827, 'round_code': 'b'},
                         {'raised_amount': 9606464, 'round_code': 'c'}]}
{'_id': ObjectId('61167af198608fe75d514780'),
 'rounds_over_8000000': [{'raised_amount': 9193583, 'round_code': 'a'}]}
{'_id': ObjectId('61167af198608fe75d514782'),
 'rounds_over_8000000': [{'raised_amount': 8544121, 'round_code': 'a'}]}
{'_id': ObjectId('61167af198608fe75d514783'),
 'rounds_over_8000000': [{'raised_amount': 9988785, 'round_code': 'a'},
                         {'raised_amount': 8918466, 'round_code': 'd'},
                         {'raised_amount': 8302021, 'round_code': 'e'}]}


#### \\$slice ,\$arrayElemAt

In [82]:
#airbnb amenities가 제공된다면,
#그 첫번째 요소를 가져와보자.
#$arrayElemAt

for i in db.airbnb.aggregate([{"$match":{"amenities":{"$exists":True}}},
                             {"$project":{"First_amenity_provided":{"$arrayElemAt":["$amenities",0]}}},
                             {"$limit":2}]):
    pprint(i)

{'First_amenity_provided': 'Internet', '_id': '10021707'}
{'First_amenity_provided': 'Wifi', '_id': '10009999'}


In [85]:
#airbnb amenities가 제공되고, 그게 5개 이상이면 제공되는 amenities의 두번째부터 3개를 가져와보자.
#$slice

for i in db.airbnb.aggregate([{"$match":{"$expr":{"$gte":[{"$size":"$amenities"},5]}}},
                             {"$project":{"second_to_firth_amenity":{"$slice":["$amenities",1,3]}}},
                             {"$limit":2}]):
    pprint(i)

{'_id': '10021707',
 'second_to_firth_amenity': ['Wifi', 'Air conditioning', 'Kitchen']}
{'_id': '10009999',
 'second_to_firth_amenity': ['Wheelchair accessible',
                             'Kitchen',
                             'Free parking on premises']}


#### accumulator

In [89]:
#fund에서 최고값만 가져와보자.
for i in db.funds.aggregate([{"$project":{"Largest_fund":{"$max":"$funding_rounds.raised_amount"}}}]):
    pprint(i)

{'Largest_fund': 9297130, '_id': ObjectId('61167af198608fe75d51477e')}
{'Largest_fund': 9737827, '_id': ObjectId('61167af198608fe75d51477f')}
{'Largest_fund': 9193583, '_id': ObjectId('61167af198608fe75d514780')}
{'Largest_fund': 7506110, '_id': ObjectId('61167af198608fe75d514781')}
{'Largest_fund': 8544121, '_id': ObjectId('61167af198608fe75d514782')}
{'Largest_fund': 9988785, '_id': ObjectId('61167af198608fe75d514783')}
{'Largest_fund': 7343947, '_id': ObjectId('61167af198608fe75d514784')}


In [90]:
#fund에서 펀딩 평균을 가져와보자.
for i in db.funds.aggregate([{"$project":{"Largest_fund":{"$avg":"$funding_rounds.raised_amount"}}}]):
    pprint(i)

{'Largest_fund': 7216487.833333333, '_id': ObjectId('61167af198608fe75d51477e')}
{'Largest_fund': 4185567.8333333335,
 '_id': ObjectId('61167af198608fe75d51477f')}
{'Largest_fund': 5183653.5, '_id': ObjectId('61167af198608fe75d514780')}
{'Largest_fund': 5220760.333333333, '_id': ObjectId('61167af198608fe75d514781')}
{'Largest_fund': 3637726.6666666665,
 '_id': ObjectId('61167af198608fe75d514782')}
{'Largest_fund': 6504653.0, '_id': ObjectId('61167af198608fe75d514783')}
{'Largest_fund': 4791044.333333333, '_id': ObjectId('61167af198608fe75d514784')}


In [91]:
#fund에서 최저값만 가져와보자.
for i in db.funds.aggregate([{"$project":{"Largest_fund":{"$min":"$funding_rounds.raised_amount"}}}]):
    pprint(i)

{'Largest_fund': 3479603, '_id': ObjectId('61167af198608fe75d51477e')}
{'Largest_fund': 509318, '_id': ObjectId('61167af198608fe75d51477f')}
{'Largest_fund': 2567725, '_id': ObjectId('61167af198608fe75d514780')}
{'Largest_fund': 2981034, '_id': ObjectId('61167af198608fe75d514781')}
{'Largest_fund': 573930, '_id': ObjectId('61167af198608fe75d514782')}
{'Largest_fund': 1878589, '_id': ObjectId('61167af198608fe75d514783')}
{'Largest_fund': 2116981, '_id': ObjectId('61167af198608fe75d514784')}


In [98]:
#fund에서 합만 가져와보자.
for i in db.funds.aggregate([{"$project":{"name":1,"Largest_fund":{"$sum":"$funding_rounds.raised_amount"},"_id":0}},{"$sort":{"Largest_fund":-1}}]):
    pprint(i)

{'Largest_fund': 43298927, 'name': 'Facebook'}
{'Largest_fund': 39027918, 'name': 'Baemin'}
{'Largest_fund': 31324562, 'name': 'Daum'}
{'Largest_fund': 31101921, 'name': 'Naver'}
{'Largest_fund': 28746266, 'name': 'Line'}
{'Largest_fund': 25113407, 'name': 'Google'}
{'Largest_fund': 21826360, 'name': 'Coupang'}
